In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from webdriver_manager.firefox import GeckoDriverManager


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import time
import random
import logging
# print(random.randint(120, 180))

In [2]:
import pandas as pd
from datetime import datetime


pos = 16
autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][pos])
senha = str(autentica['senha'][pos])


display(autentica)
name = str(autentica['campi'][pos])
data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

,login,senha,campi
0,2184579,sis1986%,CAFI
1,1882436,202104,REIPE
2,3215515,alice123,CIGR
3,3293768,nunes@123,CBJG
4,2340349,CCSA5451,CCSA
5,1239696,juliA2021,CIPJ
6,1959851,058371,CPLT
7,1966990,19891150,CGAR
8,3146148,0708@taci,CPMR
9,1257662,almox2023,CPES


In [3]:
logger = logging.getLogger('extração')
logger.setLevel(logging.DEBUG)

fHandler = logging.FileHandler(f'logs/{data_e_hora_atuais}_extracao.log', mode='w')
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fHandler.setFormatter(formatter)
logger.addHandler(fHandler)

# logging.basicConfig(level=logging.DEBUG, filename='app.log', filemode='x', format='%(asctime)s - %(levelname)s - %(message)s')
logger.info(f'iniciando extarção campus {name}')
# logger.debug('debug message')
# logger.warning('warn message')

In [4]:
pedidos = pd.read_excel('consolidacao/pedidos/2023-12-18-ajustes-finais-almox-cafi.xlsx')
# pedidos = pedidos[pedidos['acao'].str.contains('pedido no suap')]

# separando elementos de despesa
# pedidos_ed = pedidos['ed'].drop_duplicates()
# pedidos_ed.index = [x for x in range(0, len(pedidos_ed))]
# print(pedidos_ed)

# pedidos = pedidos[pedidos['ed'].str.contains(str(pedidos_ed[12]))]


# reindexando os pedidos
pedidos.index = [x for x in range(1, len(pedidos)+1)]
pedidos['i'] = [x for x in range(1, len(pedidos)+1)]
# pedidos = pedidos.iloc[0:6]
# display(pedidos)
# display(pedidos.info())

dupl = pedidos.drop_duplicates()
display(dupl.info())
pedidos = pedidos.rename(columns={"Código": "codigo", "Quantidade": "diferenca"})
display(pedidos.info())
qtd = pedidos[['i', 'diferenca']]
dict_qtd = qtd.to_dict('records')
ped = pedidos[['i', 'codigo']]
dict_ped = ped.to_dict('records')

indisponivel = "indisponivel"

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 1 to 55
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Código      55 non-null     object
 1   Nome        55 non-null     object
 2   Quantidade  55 non-null     int64 
 3   i           55 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 2.1+ KB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 1 to 55
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   codigo     55 non-null     object
 1   Nome       55 non-null     object
 2   diferenca  55 non-null     int64 
 3   i          55 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 2.1+ KB


None

In [5]:

def abrir_suap():
    
    global nav
    nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())
#     nav = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     nav = webdriver.Chrome(ChromeDriverManager().install())
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    # inserindo usuário
    logger.info('inserindo nome e senha')
    nav.find_element(By.XPATH, '//*[@id="id_username"]').click()
    nav.find_element(By.XPATH, '//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element(By.XPATH, '//*[@id="id_password"]').click()
    nav.find_element(By.XPATH, '//*[@id="id_password"]').send_keys(senha)

    nav.find_element(By.XPATH, '//*[@id="ok"]').click()

In [6]:
df_elemento_despesa = pd.DataFrame()
df_temp = pd.DataFrame(columns=('value', 'ed'))
df_temp.loc[0] = ['14', 'esportes']
# display(df_temp)

opcoes = []
valores = []

def varrer_ed():
#     abrir_suap()
    nav.get("https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/")
        
    valor_ed = nav.find_element(By.ID, 'id_elemento_despesa')
    options = [x for x in valor_ed.find_elements(By.TAG_NAME, 'option')]
    
    for i in options:
        # print(i.get_attribute('value')+" "+i.text)
        opcoes.append(i.text)
        valores.append(i.get_attribute('value'))
#         print(f'{i.text}-{i.get_attribute("value")}')

    df_elemento_despesa['value'] = valores
    df_elemento_despesa['ed']= opcoes
    
#     display(df_elemento_despesa)

In [7]:
def get_qtd():
    
    logger.info('abrindo suap')
    abrir_suap()
    print(f'Extração iniciada as: {datetime.now()}')
    
    logger.info('criando tabela')
    df_tabela = pd.DataFrame()
    tabela = []
    codigos = []
    unidades = []
    descricoes = []
    quantidades = []
    valores = []
    elemento = []
    
    try:
        pag = 1
        saida = True
        while saida:
            logger.debug(f'recolhendo dados da pagina {pag}')
            nav.get(f'https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/?page={pag}&material=&elemento_despesa=&estoque_tipo=emcampus&unidade_medida=')
            celula = nav.find_element(By.TAG_NAME, 'td')
                
            if celula:
                lista = nav.find_element(By.TAG_NAME, 'table')
                td = lista.find_elements(By.TAG_NAME, 'td')
                
                for i in td:
                    tabela.append(i.text)
                
                pag += 1
            else:
                saida = False
                
    except NoSuchElementException:
        logger.warning('fim da paginação')
        print('fim da paginação')
    
    logger.debug('iniciando fatiamento da tabela')
    for cod in range(0, len(tabela), 7):
        codigos.append(tabela[cod])

    for des in range(1, len(tabela), 7):
        descricoes.append(tabela[des])
        
    for uni in range(2, len(tabela), 7):
        unidades.append(tabela[uni])

    for qtd in range(4, len(tabela), 7):
        quantidades.append(tabela[qtd])
    
    for val in range(5, len(tabela), 7):
        valores.append(tabela[val])
        
    for ed in range(3, len(tabela), 7):
        elemento.append(tabela[ed])

    logger.debug('montando DataFrame')
    
    df_tabela['codigo'] = codigos
    df_tabela['descricao'] = descricoes
    df_tabela['und'] = unidades
    
    df_tabela['qtd'] = quantidades
    df_tabela['qtd'] = pd.to_numeric(df_tabela['qtd'])
    
    df_tabela['vlr_total'] = valores
    df_tabela['vlr_total'] = df_tabela['vlr_total'].str[2:]
    df_tabela['vlr_total'] = pd.to_numeric(df_tabela['vlr_total'])
    
    df_tabela['vlr_unt'] = df_tabela['vlr_total'] / df_tabela['qtd']
    df_tabela['vlr_unt'] = df_tabela['vlr_unt'].round(2)
    
    df_tabela['ed'] = elemento
    
    print(f'Extração finalizada as: {datetime.now()}')
    print(df_tabela.info())
    
    logger.debug('exportando arquivo .xlsx')
    
    df_tabela.to_excel(f'relatorio/{data_e_hora}_{name}_relatorio_almox.xlsx', index=False)
#     display(df_tabela)
    
    nav.quit()
    
logger.info(f'Extarção campus {name} finalizada')

In [8]:
def set_pedidos():
    abrir_suap()

    nav.get('https://suap.ifpe.edu.br/almoxarifado/form_requisicao_pessoa_pedido/')

    # preenchendo campo do requisitante
    logger.info('preenchendo campo do requisitante ')

    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/fieldset/div[2]/input[1]').send_keys('Andrea ferreira')
    time.sleep(3)
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/fieldset/div[2]/input[1]').send_keys(Keys.ENTER)
    time.sleep(3)
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/fieldset/div[2]/input[1]').send_keys(Keys.ARROW_DOWN)
    time.sleep(3)
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/fieldset/div[2]/input[1]').send_keys(Keys.ENTER)

    # preenchento campo do pedido do material 
    logger.info('preenchento campo do pedido do material ')
    for i in dict_ped:
        logger.info(f'digitando código do material {i["codigo"]}')
        nav.find_element(By.XPATH, f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[{i["i"]}]/input[1]').send_keys(i['codigo'])
        time.sleep(3)
        nav.find_element(By.XPATH, f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[{i["i"]}]/input[1]').send_keys(Keys.ENTER)
        time.sleep(3)
        nav.find_element(By.XPATH, f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[{i["i"]}]/input[1]').send_keys(Keys.ARROW_DOWN)
        time.sleep(3)
        nav.find_element(By.XPATH, f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[{i["i"]}]/input[1]').send_keys(Keys.ENTER)
        time.sleep(1)
        nav.find_element(By.NAME, 'addItem').click()

    for i in dict_qtd:    
        logger.info(f'digitando quantidades dos pedidos {i["diferenca"]}')
        nav.find_element(By.XPATH, f'/html/body/div[1]/div[3]/form/div[1]/fieldset/div[{i["i"]}]/input[3]').send_keys(i['diferenca'])

In [9]:
get_qtd()

[WDM] - Downloading: 19.9kB [00:00, 3.79MB/s]                   
[WDM] - Downloading: 100%|██████████| 3.10M/3.10M [00:00<00:00, 12.0MB/s]


Extração iniciada as: 2024-01-17 16:36:15.185684
fim da paginação
Extração finalizada as: 2024-01-17 16:37:00.970693
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 718 entries, 0 to 717
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   codigo     718 non-null    object 
 1   descricao  718 non-null    object 
 2   und        718 non-null    object 
 3   qtd        718 non-null    int64  
 4   vlr_total  718 non-null    float64
 5   vlr_unt    718 non-null    float64
 6   ed         718 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 39.4+ KB
None
